In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.stats import norm
np.set_printoptions(precision=5, suppress=True)

IvyDB contains a complete historical record of end-of-day data on all US exchange-traded equity and index options (including options on ETFs and ADRs) from January 1996 onward. The data includes both daily option pricing information (symbol, date, closing bid and ask quote, volume, and open interest) as well as high, low, and closing prices for the underlying equity or index. IvyDB also provides all interest rate, dividend, and corporate action information for each security, so you can correlate your own option pricing models with calculations.



In [52]:
# load SPX options data
price_df = pd.read_csv('211c7adb142fa896.csv', parse_dates=['date'])
df = pd.read_csv('0b9c74e2057156a8.csv', parse_dates=['date', 'exdate', 'last_date'])
df = pd.merge(df, price_df[['date', 'close']])

derivs = df[(df['date'] == '2019-06-03') & (df['exdate'] == '2019-06-07')]
price = derivs.close.iloc[0]
derivs = derivs[derivs.volume > 10]
derivs = derivs[np.abs(derivs.strike_price / 1000 - price) / price < .3]

calls = derivs[derivs.cp_flag == 'C']
puts = derivs[derivs.cp_flag == 'P']
p = np.arange(1, 6000, 1)
print (p.min(), p.max(), p.size, price)

calls_bid = np.array(calls.best_bid)
calls_ask = np.array(calls.best_offer)
calls_strike = np.array(calls.strike_price / 1000)

puts_bid = np.array(puts.best_bid)
puts_ask = np.array(puts.best_offer)
puts_strike = np.array(puts.strike_price / 1000)

puts_strike.size, calls_strike.size

1 5999 5999 2744.45


(108, 49)

In [53]:
names = ["buy put %.3f for %.3f" % (s, p) for p, s in zip(puts_ask, puts_strike)] + \
        ["write put %.3f for %.3f" % (s, p) for p, s in zip(puts_bid, puts_strike)] + \
        ["buy call %.3f for %.3f" % (s, p) for p, s in zip(calls_ask, calls_strike)] + \
        ["write call %.3f for %.3f" % (s, p) for p, s in zip(calls_bid, calls_strike)] + \
        ["buy underlying", "short underlying"]

In [54]:
option_fee = .65/100
short_fee = .0003
long_fee = .0003
P = np.hstack([
    np.maximum(puts_strike[None, :] - p[:, None], 0) - option_fee,
    -np.maximum(puts_strike[None, :] - p[:, None], 0) - option_fee,
    np.maximum(p[:, None] - calls_strike[None, :], 0) - option_fee,
    -np.maximum(p[:, None] - calls_strike[None, :], 0) - option_fee,
    p[:, None],
    -p[:, None]
])
d = np.concatenate([
    puts_ask,
    -puts_bid,
    calls_ask,
    -calls_bid,
    np.array([price + long_fee * price]),
    np.array([-price + short_fee * price])
])
m, n = P.shape

In [55]:
# check if arbitrage exists
R = P - d[None,:]
x = cp.Variable(R.shape[1])
prob = cp.Problem(cp.Minimize(0), [x >= 0, cp.min(R @ x) >= 1])
assert prob.solve() == np.inf, "Arbitrage exists"

## Maximum entropy / min dirichlet energy, total variation

In [ ]:
pi = cp.Variable(m)

dirichlet = cp.sum_squares(cp.diff(pi))
entr = cp.sum(cp.entr(pi))
tv = cp.norm1(cp.diff(pi, k=1))

cons = [P.T @ pi <= d, cp.sum(pi) == 1, pi >= 0]

plt.close()
cp.Problem(cp.Minimize(dirichlet), cons).solve(solver=cp.MOSEK)
plt.plot(p, pi.value, label='entr', c='k')
plt.xlabel("$p$")
plt.savefig("../price_distr_derivs/figs/dirichlet.pdf")

plt.close()
cp.Problem(cp.Maximize(entr), cons).solve(solver=cp.MOSEK)
pi_entr = np.copy(pi.value)
plt.plot(p, pi.value, label='entr', c='k')
plt.xlabel("$p$")
plt.savefig("../price_distr_derivs/figs/entr.pdf")

plt.close()
cp.Problem(cp.Minimize(tv), cons).solve(solver=cp.MOSEK)
plt.plot(p, pi.value, label='entr', c='k')
plt.xlabel("$p$")
plt.savefig("../price_distr_derivs/figs/tv.pdf")

## Closest log-normal distribution

In [ ]:
pilognormal = np.ones(m) / m
for k in range(5):
    mu = pilognormal @ np.log(p)
    sigma = np.sqrt(pi_entr @ np.square(np.log(p) - mu))
    nu = np.append(0, np.diff(norm.cdf(np.log(p), loc=mu, scale=sigma)))
    nu /= nu.sum()
    result = cp.Problem(cp.Minimize(cp.sum(cp.kl_div(pi, nu))), cons).solve(solver=cp.MOSEK)
    pilognormal = pi.value
    print (k, result, mu, sigma)
mu = pilognormal @ np.log(p)
sigma = np.sqrt(pi_entr @ np.square(np.log(p) - mu))
print ("mu: %.3f, sigma: %.3f" % (mu, sigma))
plt.close()
plt.plot(p, pilognormal, c='black')
plt.savefig("../price_distr_derivs/figs/lognormal.pdf")
plt.show()

In [ ]:
annualized_volatility = np.std(np.prod(np.exp(np.random.normal(loc=mu, scale=sigma, size=(1_000_000, int(365/36)))) / price, axis=1))
annualized_volatility

## Lower and upper bounds on the mean

In [ ]:
result = cp.Problem(cp.Maximize(p @ pi), cons).solve(solver=cp.MOSEK)
print ("lower bound:", result)
result = cp.Problem(cp.Minimize(p @ pi), cons).solve(solver=cp.MOSEK)
print ("upper bound:", result)

## Upper/lower bounds on CDF

In [ ]:
upper_bounds = []
lower_bounds = []
for i in np.arange(0, m, 50):
    try:
        upper_bound = cp.Problem(cp.Maximize(cp.sum(pi[:i])), cons).solve(solver=cp.MOSEK)
    except:
        upper_bound = np.nan
    upper_bounds.append(upper_bound)
    try:
        lower_bound = cp.Problem(cp.Minimize(cp.sum(pi[:i])), cons).solve(solver=cp.MOSEK)
    except:
        lower_bound = np.nan
    lower_bounds.append(lower_bound)

In [ ]:
plt.close()
plt.figure(figsize=(8,4))
plt.plot(p[::50], upper_bounds, c='black', label='upper')
plt.plot(p[::50], lower_bounds, '--', c='black', label='lower')
plt.axvline(price, c='gray', label='price')
plt.xlim(0,20000)
plt.ylim(.0001)
plt.ylim(5e-3)
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$\mathbf{Prob}(p \leq x)$')
plt.savefig('../price_distr_derivs/figs/cdf_bounds.pdf')
plt.show()

## Upper/lower bounds on VaR

In [ ]:
plt.close()
plt.semilogx(lower_bounds, (p[::10] - price) / price, c='black', label='upper')
plt.semilogx(upper_bounds, (p[::10] - price) / price, '--', c='black', label='lower')
plt.legend()
plt.xlim(.00001, 1)
plt.xlabel('$\epsilon$')
plt.ylabel('$\mathbf{VaR}(p, \epsilon)$')
plt.savefig('../price_distr_derivs/figs/var_bounds.pdf')

## Upper bound on CVaR

In [ ]:
cvar_upper = []
epss = np.logspace(-4,-.1,25)
for eps in epss:
    t = cp.Variable(1)
    result = cp.Problem(cp.Maximize(t), cons + [p[i] + pi @ np.maximum(p - p[i], 0) / (1 - eps) >= t for i in range(m)]).solve(solver=cp.MOSEK)
    cvar_upper.append(result)

In [ ]:
plt.semilogx(np.logspace(-4,-.1,25), (np.array(cvar_upper) - price) / price)
plt.semilogx(upper_bounds, (p[::10] - price) / price, '--', c='black', label='lower')
plt.xlim(1e-4,10**(-.1))

## Upper/lower bounds on complementary CDF

In [ ]:
upper_bounds = []
lower_bounds = []
for i in np.arange(0, m, 10):
    try:
        upper_bound = cp.Problem(cp.Maximize(cp.sum(pi[i:])), cons).solve(solver=cp.MOSEK)
    except:
        upper_bound = np.nan
    upper_bounds.append(upper_bound)
    try:
        lower_bound = cp.Problem(cp.Minimize(cp.sum(pi[i:])), cons).solve(solver=cp.MOSEK)
    except:
        lower_bound = np.nan
    lower_bounds.append(lower_bound)

In [ ]:
plt.close()
plt.semilogy((p[::10] - price) / price, upper_bounds, c='black', label='upper')
plt.semilogy((p[::10] - price) / price, lower_bounds, '--', c='black', label='lower')
plt.axvline(0, c='gray', label='price')
plt.xlim(-.2, .2)
plt.ylim(.0001)
plt.legend()
plt.xlabel('a')
plt.ylabel('prob(return >= a)')
plt.savefig('../price_distr_derivs/figs/ccdf_bounds.pdf')

In [ ]:
plt.semilogx(upper_bounds, (p[::10] - price) / price, c='black', label='upper')
plt.semilogx(lower_bounds, (p[::10] - price) / price, '--', c='black', label='lower')
plt.axvline(0, c='blue', label='price')
plt.xlim(.0001, 1)
plt.legend()
plt.xlabel('$\epsilon$')
plt.ylabel('$\mathbf{VaR}(-p,\epsilon)$')

## Bounds on costs

In [ ]:
ask_lower_bounds = []
bid_upper_bounds = []
for i in range(puts_ask.size):
    idx = np.ones(n, dtype=bool)
    idx[i] = False
    idx[puts_ask.size + i] = False
    pi = cp.Variable(m)
    cbuy = cp.Variable()
    csell = cp.Variable()
    
    cons = [pi >= 0, cp.sum(pi) == 1, P[:,idx].T @ pi <= d[idx], csell <= cbuy,
            P[:,~idx].T @ pi <= cp.vstack([cbuy, -csell])[:,0]]
    ask_lower_bound = cp.Problem(cp.Minimize(cbuy), cons).solve(solver=cp.MOSEK)
    bid_upper_bound = cp.Problem(cp.Maximize(csell), cons).solve(solver=cp.MOSEK)
    
    ask_lower_bounds.append(ask_lower_bound)
    bid_upper_bounds.append(bid_upper_bound)
    assert puts_ask[i] >= ask_lower_bound
    assert puts_bid[i] <= bid_upper_bound
    print ("Put", i)
    print ("ask %3.3f bid %3.3f" % (puts_ask[i], puts_bid[i]))
    print ("ask lower bound %3.3f" % (ask_lower_bound))
    print ("bid upper bound %3.3f" % (bid_upper_bound))

In [ ]:
plt.close()
plt.plot(puts_strike[idx], (puts_ask - np.array(ask_lower_bounds))[idx], c='k')
plt.xlabel('strike price')
plt.ylabel('distance to lower bound')
plt.savefig('../price_distr_derivs/figs/ask_bounds.pdf')

In [ ]:
idx = np.argsort(puts_strike)

In [ ]:
plt.plot(puts_strike[idx], (np.array(bid_upper_bounds) - puts_bid)[idx], c='k')
plt.xlabel('strike price')
plt.ylabel('distance to upper bound')
plt.savefig('../price_distr_derivs/figs/bid_bounds.pdf')

## Bounds on the price of a new investment

In [ ]:
# a binary option that pays 1 if price stays within +/- 2%, and 0 otherwise
pi = cp.Variable(m)
c = cp.Variable()

cons = [pi >= 0, cp.sum(pi) == 1, P[:,idx].T @ pi <= d[idx], (np.abs(p - price) / price < .02).astype(np.float) @ pi == c]
lower = cp.Problem(cp.Minimize(c), cons).solve(solver=cp.MOSEK)
upper = cp.Problem(cp.Maximize(c), cons).solve(solver=cp.MOSEK)
lower, upper